In [3]:
# %%capture
!pip install gym seaborn keras-rl2 mlflow[extras] matplotlib

In [4]:
from episode_runner import ep_run

In [36]:
# training
reward, dqn = ep_run(
    reward_factor=0.2, plot_title="Training", train=True
).run_experimentation(run_name="RUN_NAME", lr=1e-3, num_steps=200000)

In [ ]:
# testing
test_ep_runner = ep_run(
    reward_factor=0.2, plot_title="testing", train=False
)  # lockdown
test_reward = dqn.test(test_ep_runner.env, num_episodes=100, visualize=False)

In [ ]:
# manual simulation
ep_runner = ep_run(plot_title="continuous restrictions", train=False)  # lockdown
simulation_reward = ep_runner.test(
    test_ep_runner.env, experiment_type="lockdown", num_episodes=1
)